In [ ]:
import json
from pprint import pprint

#log_file = 'serviceslogs.json'
#input_service_name='myca'

log_file =  input('Please enter the log file name (e.g. serviceslogs.json):')
input_service_name =  input('Please enter the service name (e.g. myca), or leave blank for all):')
if not input_service_name:  
    input_service_name = None

def open_file(input_file):
    with open(input_file, encoding='utf-8') as f:
        return json.load(f)
    
def parse_file(input_file):
    set_service_name = set()
    set_severity = set()
    list_service_severity = []

    for entry in input_file:
        if "resource" in entry and "labels" in entry["resource"] and "service_name" in entry["resource"]["labels"]:
            service_name = entry["resource"]["labels"]["service_name"]
            if "severity" in entry:
                severity = entry["severity"]
            else:
                severity = None
            if service_name != '':
                set_service_name.add(service_name)
            if severity:
                set_severity.add(severity)

            found = False
            for record in list_service_severity:
                if record["service_name"] == service_name and record["severity"] == severity:
                    record["count"] += 1
                    found = True
                    break
            if not found and severity and service_name != '':
                list_service_severity.append({"service_name": service_name, "severity": severity, "count": 1})
    return set_service_name, set_severity, list_service_severity

def generate_output(set_service_name, set_severity, list_service_severity, given_service_name=None):
    max_service_name_length = max(len(name) for name in set_service_name)
    max_count = max(record["count"] for record in list_service_severity)
    max_count_length = len(str(max_count))
    output_lines = []
    if given_service_name:
        set_service_name = {given_service_name}

    for service_name in set_service_name:
        output = f'{service_name:<{max_service_name_length}}' 
        sorted_severities = sorted(set_severity)
        for i, severity in enumerate(sorted_severities):
            #print(list_service_severity[service_name][severity])
            for record in list_service_severity:
                if record["service_name"] == service_name and record["severity"] == severity:
                    count = record["count"]
                    break
                else:
                    count = 0
            output += f' {severity}: {count:{max_count_length}d}'
            if i < len(sorted_severities) - 1:
                output += '/'
        output_lines.append(output)
    return '\n'.join(output_lines)


json_log = open_file(log_file)
set_service_name, set_severity, list_service_severity = parse_file(json_log)  
output_result  = generate_output(set_service_name, set_severity, list_service_severity, given_service_name=input_service_name)

print('--------------------------------OUTPUT--------------------------------')
print(output_result)
